# Pipeline built-in filters

本文介紹常用的內建 filter。

In [1]:
import os 
import pandas as pd
import numpy as np 
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw"
os.environ['TEJAPI_KEY'] = "your key"
os.environ['mdate'] = "20140702 20220702"
os.environ['ticker'] = "IR0001 2330 2454 1101 1301 1303 2337 2451 2388 2002 2903 2881 2885 2890 1802 2101 3711"
!zipline ingest -b tquant

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\Scripts\zipline.exe\__main__.py", line 4, in <module>
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\zipline\__init__.py", line 21, in <module>
    from zipline.utils.calendar_utils import get_calendar
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\zipline\utils\calendar_utils.py", line 24, in <module>
    from exchange_calendars import ExchangeCalendar as TradingCalendar
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\exchange_calendars\__init__.py", line 16, in <module>
    from .calendar_utils import (
  File "C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\exchange_calen

In [2]:
from zipline.master import run_pipeline
from zipline.pipeline import Pipeline
from zipline.pipeline.data import TWEquityPricing

start = pd.Timestamp("2018-02-06", tz='utc')
end = pd.Timestamp("2022-02-06", tz='utc')

<span id="menu"></span>
    
## 選單
* [All](#All)
* [Any](#Any)
* [AtLeastN](#AtLeastN)
* [AllPresent](#AllPresent)
* [StaticAssets](#StaticAssets)
* [StaticSids](#StaticSids)
* [SingleAsset](#SingleAsset)

<span id="All"></span>

### zipline.pipeline.filters.All

若在 n 日內，若一資產每日皆為 True，該資產為 True。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.Boundcolumn_ or _boolean_
        資產價量資訊或條件值。
* window_length: _int_
        決定 n 日。

[回到選單](#menu)

In [3]:
from zipline.pipeline.filters import All
def make_pipeline():
    return Pipeline(
        columns = {
            "ALL": All(
                inputs = [TWEquityPricing.close.latest > 40], # 設定條件為前一日收盤價 > 40 時為 True
                window_length = 1
            )
        }
    )

run_pipeline(make_pipeline(), start, end)

ALL
2018-02-06 00:00:00+00:00 Equity(0 [1101])     False
                          Equity(1 [1301])      True
                          Equity(2 [1303])      True
                          Equity(3 [1802])     False
                          Equity(4 [2002])     False
...                                              ...
2022-01-26 00:00:00+00:00 Equity(12 [2885])    False
                          Equity(13 [2890])    False
                          Equity(14 [2903])    False
                          Equity(15 [3711])     True
                          Equity(16 [IR0001])   True

[16456 rows x 1 columns]

<span id="Any"></span>

### zipline.pipeline.filters.Any

若在 n 日內，若一資產任一日為 True，該資產為 True。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.Boundcolumn_ or _boolean_
        資產價量資訊或條件值。
* window_length: _int_
        決定 n 日。

[回到選單](#menu)

In [4]:
from zipline.pipeline.filters import Any
def make_pipeline():
    return Pipeline(
        columns = {
            "Any": Any(
                inputs = [TWEquityPricing.close.latest > 40], # 設定條件為前一日收盤價 > 40 時為 True
                window_length = 10
            )
        }
    )

run_pipeline(make_pipeline(), start, end)

Any
2018-02-06 00:00:00+00:00 Equity(0 [1101])     False
                          Equity(1 [1301])      True
                          Equity(2 [1303])      True
                          Equity(3 [1802])     False
                          Equity(4 [2002])     False
...                                              ...
2022-01-26 00:00:00+00:00 Equity(12 [2885])    False
                          Equity(13 [2890])    False
                          Equity(14 [2903])    False
                          Equity(15 [3711])     True
                          Equity(16 [IR0001])   True

[16456 rows x 1 columns]

<span id="AtLeastN"></span>

### zipline.pipeline.filters.AtLeastN

若在 m 日內，若一資產有 n 日為 True，該資產為 True。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.Boundcolumn_ or _boolean_
        資產價量資訊或條件值。
* window_length: _int_
        決定 m 日。
* N: _int_
        決定 n 日。

[回到選單](#menu)

In [5]:
from zipline.pipeline.filters import AtLeastN
def make_pipeline():
    return Pipeline(
        columns = {
            "AtLeastN": AtLeastN(
                inputs = [TWEquityPricing.close.latest > 40], # 設定條件為前一日收盤價 > 40 時為 True
                window_length = 10,
                N = 2
            )
        }
    )

run_pipeline(make_pipeline(), start, end)

AtLeastN
2018-02-06 00:00:00+00:00 Equity(0 [1101])        False
                          Equity(1 [1301])         True
                          Equity(2 [1303])         True
                          Equity(3 [1802])        False
                          Equity(4 [2002])        False
...                                                 ...
2022-01-26 00:00:00+00:00 Equity(12 [2885])       False
                          Equity(13 [2890])       False
                          Equity(14 [2903])       False
                          Equity(15 [3711])        True
                          Equity(16 [IR0001])      True

[16456 rows x 1 columns]

<span id="AllPresent"></span>

### zipline.pipeline.filters.AllPresent

若在 n 日內，每日皆有資料，該資產為 True。

#### Parameters:
* inputs: _zipline.pipeline.data.Dataset.Boundcolumn_
        資產價量資訊或條件值。
* window_length: _int_
        決定 n 日。

[回到選單](#menu)

In [6]:
from zipline.pipeline.filters import AllPresent
def make_pipeline():
    return Pipeline(
        columns = {
            "AllPresent": AllPresent(
                inputs = [TWEquityPricing.close], 
                window_length = 10
            )
        }
    )

run_pipeline(make_pipeline(), start, end).loc["2018-05-04"]
# 可注意到 3711 在 2018-04-30 才上市，因此 2018-05-04 為 False

AllPresent
2018-05-04 00:00:00+00:00 Equity(0 [1101])           True
                          Equity(1 [1301])           True
                          Equity(2 [1303])           True
                          Equity(3 [1802])           True
                          Equity(4 [2002])           True
                          Equity(5 [2101])           True
                          Equity(6 [2330])           True
                          Equity(7 [2337])           True
                          Equity(8 [2388])           True
                          Equity(9 [2451])           True
                          Equity(10 [2454])          True
                          Equity(11 [2881])          True
                          Equity(12 [2885])          True
                          Equity(13 [2890])          True
                          Equity(14 [2903])          True
                          Equity(15 [3711])         False
                          Equity(16 [IR0001])        True

In [7]:
# 首先抓出所有下載的股價
from zipline.data import bundles
bundle = bundles.load('tquant')
sids = bundle.asset_finder.equities_sids
assets = bundle.asset_finder.retrieve_all(sids)
assets

[Equity(0 [1101]),
 Equity(1 [1301]),
 Equity(2 [1303]),
 Equity(3 [1802]),
 Equity(4 [2002]),
 Equity(5 [2101]),
 Equity(6 [2330]),
 Equity(7 [2337]),
 Equity(8 [2388]),
 Equity(9 [2451]),
 Equity(10 [2454]),
 Equity(11 [2881]),
 Equity(12 [2885]),
 Equity(13 [2890]),
 Equity(14 [2903]),
 Equity(15 [3711]),
 Equity(16 [IR0001])]

<span id="StaticAssets"></span>

### zipline.pipeline.filters.StaticAssets

指定特定資產為 True。

#### Parameters:
* assets: _iterable[zipline.assets.Asset]_
        指定資產。


[回到選單](#menu)

In [8]:
from zipline.pipeline.filters import StaticAssets
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    return Pipeline(
        columns = {
            "StaticAssets": StaticAssets(
                assets = assets[4:8]
            )
        }
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start,
    end = end,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [2881])           False
Equity(12 [2885])           False
Equity(13 [2890])           False
Equity(14 [2903])           False
Equity(16 [IR0001])         False
                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [288

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [2881])           False
Equity(12 [2885])           False
Equity(13 [2890])           False
Equity(14 [2903])           False
Equity(15 [3711])           False
Equity(16 [IR0001])         False
                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [245

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [2881])           False
Equity(12 [2885])           False
Equity(13 [2890])           False
Equity(14 [2903])           False
Equity(15 [3711])           False
Equity(16 [IR0001])         False
                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [245

...

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages

                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [2881])           False
Equity(12 [2885])           False
Equity(13 [2890])           False
Equity(14 [2903])           False
Equity(15 [3711])           False
Equity(16 [IR0001])         False
                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [245

                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [2454])           False
Equity(11 [2881])           False
Equity(12 [2885])           False
Equity(13 [2890])           False
Equity(14 [2903])           False
Equity(15 [3711])           False
Equity(16 [IR0001])         False
                     StaticAssets
Equity(0 [1101])            False
Equity(1 [1301])            False
Equity(2 [1303])            False
Equity(3 [1802])            False
Equity(4 [2002])             True
Equity(5 [2101])             True
Equity(6 [2330])             True
Equity(7 [2337])             True
Equity(8 [2388])            False
Equity(9 [2451])            False
Equity(10 [245

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

<span id="StaticSids"></span>

### zipline.pipeline.filters.StaticSids

指定特定資產為 True。

#### Parameters:
* sids: _iterable[int]_
        指定資產的 sid。


[回到選單](#menu)

In [9]:
from zipline.pipeline.filters import StaticSids
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    return Pipeline(
        columns = {
            "StaticSids": StaticSids(
                sids = range(4,8)
            )
        }
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start,
    end = end,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(1

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(15 [3711])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(1

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(15 [3711])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(1

...

                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(15 [3711])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(1

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(15 [3711])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(1

                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(12 [2885])         False
Equity(13 [2890])         False
Equity(14 [2903])         False
Equity(15 [3711])         False
Equity(16 [IR0001])       False
                     StaticSids
Equity(0 [1101])          False
Equity(1 [1301])          False
Equity(2 [1303])          False
Equity(3 [1802])          False
Equity(4 [2002])           True
Equity(5 [2101])           True
Equity(6 [2330])           True
Equity(7 [2337])           True
Equity(8 [2388])          False
Equity(9 [2451])          False
Equity(10 [2454])         False
Equity(11 [2881])         False
Equity(1

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

<span id="SingleAsset"></span>

### zipline.pipeline.filters.SingleAsset

指定單一特定資產為 True。

#### Parameters:
* asset: _zipline.assets.Asset_
        指定資產。

[回到選單](#menu)

In [10]:
from zipline.pipeline.filters import SingleAsset
from zipline import run_algorithm
from zipline.api import symbol, attach_pipeline, pipeline_output

def make_pipeline():
    return Pipeline(
        columns = {
            "SingleAsset": SingleAsset(
                asset = assets[4]
            )
        }
    )

def initialize(context):
    my_pipe = attach_pipeline(make_pipeline(), 'my_pipe')
    
def handle_data(context, data):
    pipe = pipeline_output('my_pipe')
    print("=" * 100)
    print(pipe)

def analyze(context, perf):
    pass

results = run_algorithm(
    start = start,
    end = end,
    initialize = initialize,
    capital_base = 1e6,
    handle_data = handle_data,
    analyze = analyze, 
    bundle = 'tquant'
)


                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(15 [3711])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(15 [3711])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 

...

                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(15 [3711])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en

                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(15 [3711])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 

                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 [2881])          False
Equity(12 [2885])          False
Equity(13 [2890])          False
Equity(14 [2903])          False
Equity(15 [3711])          False
Equity(16 [IR0001])        False
                     SingleAsset
Equity(0 [1101])           False
Equity(1 [1301])           False
Equity(2 [1303])           False
Equity(3 [1802])           False
Equity(4 [2002])            True
Equity(5 [2101])           False
Equity(6 [2330])           False
Equity(7 [2337])           False
Equity(8 [2388])           False
Equity(9 [2451])           False
Equity(10 [2454])          False
Equity(11 

C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\zipline-tej\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value en